In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm
from N_plotting_utils import *
from N_ranking_utils import *
from sklearn.metrics.pairwise import *
import sklearn
import scipy
from sklearn.decomposition import PCA

In [3]:
def feature_values_boxplot(all_features, features_to_use, features_to_plot=None,ax=None): 
    features_preprocessed=pd.DataFrame()
    for f in features_to_use:
        feat=all_features[f]
        #feat=feat.query('benchmark in ["bbob","random"]')
        to_plot,_=preprocess_ela(feat.drop(columns=['benchmark']),[])
        to_plot_scaled=pd.DataFrame(MinMaxScaler().fit_transform(to_plot), columns=to_plot.columns, index=to_plot.index)
        if features_to_plot is not None:
            to_plot_scaled=to_plot_scaled[features_to_plot]
        to_plot_scaled['benchmark']=[benchmark_name_mapping(b) for b in feat['benchmark'].values]
        t=to_plot_scaled.reset_index().melt(['f','benchmark'], var_name='feature name', value_name='scaled feature value')
        t['features']=f
        features_preprocessed=pd.concat([features_preprocessed, t])
        
    
    ax=sns.catplot(features_preprocessed, y='feature name',x='scaled feature value',hue='benchmark',ax=ax,palette=color_palette_4, col='features',kind='box',aspect=0.5,height=20)
    #ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    #parallel_coordinates(to_plot_scaled.sample(200),class_column='benchmark',color=color_palette_4)

In [4]:
dimension=3
all_features={f'50d_sy' :get_features(dimension,1000000,feature_dir=f,sample_count_dimension_factor=50,scaled=True,drop_constant=False)[1] for f in ['ela_features'] }
all_features.update({f'50d' :get_features(dimension,1000000,feature_dir=f,sample_count_dimension_factor=50,scaled=False,drop_constant=False)[1] for f in ['ela_features'] })


20337
20337


In [5]:
all_features.keys()

dict_keys(['50d_sy', '50d'])

In [6]:
def plot_2d(feature_df,save_file=None):
    fs_to_use=list(filter(lambda x: "costs_runtime" not in x and "costs_fun_evals" not in x, feature_df.columns))
    feature_df=feature_df[fs_to_use].sort_values('benchmark')
    plt.figure(figsize=(10,10))
    
    #feature_df_scaled=MinMaxScaler().fit_transform(feature_df.drop(columns=['benchmark']))
    f2d = TSNE(2).fit_transform(feature_df.drop(columns=['benchmark']))
    f2d=pd.DataFrame(f2d, index=feature_df.index,columns=['x','y'])
    f2d['benchmark']=feature_df['benchmark'].apply(lambda x: x.upper() if x!='m4' else 'ZIGZAG')
    
    #palette={'AFFINE':'#00ff84', 'BBOB':"black", 'ZIGZAG':color_palette_4[2], 'RANDOM':color_palette_4[3]}
        
    ax=sns.scatterplot(f2d,hue='benchmark',x='x',y='y',alpha=0.4, style='benchmark')
    ax.set(xticklabels=[])
    ax.set(yticklabels=[])
    
    plt.tight_layout()
    plt.savefig(f'new_data/figures_results_ela_scaling/2d_plot/{save_file}.pdf')
    plt.show()

In [7]:
def plot_cosine(feature_df,save_file=None):
    fs_to_use=list(filter(lambda x: "costs_runtime" not in x and "costs_fun_evals" not in x, feature_df.columns))
    feature_df=feature_df[fs_to_use].sort_values('benchmark')
    plt.figure(figsize=(10,10))
    feature_df_scaled=MinMaxScaler().fit_transform(feature_df.drop(columns=['benchmark']))
    
    cs=cosine_similarity(feature_df_scaled,feature_df_scaled )
    cs=pd.DataFrame(cs, index=feature_df['benchmark'], columns=feature_df['benchmark'])

    ax=sns.heatmap(cs)
    ax.set(xticklabels=[])
    ax.set(yticklabels=[])
    
    plt.tight_layout()
    plt.savefig(f'new_data/figures_results_ela_scaling/sim_plot/{save_file}.pdf')
    plt.show()

In [8]:
def plot_features(all_features,plotting_function):
    all_feature_names=all_features['50d'].drop(columns=['benchmark']).columns
    feature_groups=list(set([cc.split('.')[0] for cc in all_feature_names]))
    print('ALL')
    plotting_function(all_features['50d'],f'dim_{dimension}_all')
    plotting_function(all_features['50d_sy'],f'dim_{dimension}_all_sy')
    for feature_group in feature_groups:
        for c in ['50d','50d_sy']:
            print(feature_group)
            features_from_group=list(filter(lambda x: x.startswith(feature_group+'.'), all_feature_names))+['benchmark']
            plotting_function(all_features[c][features_from_group],f'dim_{dimension}_{c}_{feature_group}')

In [ ]:
plot_features(all_features, plot_2d)

ALL


/opt/conda/envs/Base/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:106: RuntimeWarning: overflow encountered in cast
  distances_data = distances_data.astype(np.float32, copy=False)
